<a href="https://colab.research.google.com/github/EmelyanovAndreyNSK/PythonTasks/blob/master/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введем переменные

$bA$ - количество поставки руды A

$bB$ - количество поставки руды B

$conv$ - количество $II$ продукта отправленного на конвертацию

$sI$ - количество $I$ товара на продажу (Доля)

$sIlim$ - покупка до 4 тысяч тонн в день (Доля)

$sIunlim$ - безлимитная покупка (Доля)

$bI$ - закупка первого товара

$bII$ - закупка второго товара





# Входные данные

## Закуп:
$A\_pr = 3.25\_ руб./т.$ - стоимость покупки руды A

$A\_am = 100\_ тыс. тонн/ день$ - количество поставки руды A в день

$B\_pr = 3.40\_ руб./т.$ - стоимост покупки руды B

$B\_am =  30\_ тыс. тонн/ день $ - количество поставки руды B в день

## Переработка:

$proc\_pow = 100\_ тыс. тонн/ день$ - общая мощность основной переработки

$proc\_pr = 0.35\_ руб./т. $ стоимость общей переработки


$ A\_proc = \left\{
\begin{array}{ll}
          A\_proc1 = 0.15\: т.\: продукта\: I, & \mbox {}\\
          A\_proc2 = 0.85\: т.\: продукта\: II, & \mbox { } \\
\end{array} \right. $ - с одной тонны

$B\_proc = \left\{
\begin{array}{ll}
          B\_proc1 = 0.25\: т.\: продукта\: I, & \mbox {}\\
          B\_proc2 = 0.75\: т.\: продукта\: II, & \mbox { } \\
\end{array} \right. $ - с одной тонны

$ Conv\_proc = \left\{
\begin{array}{ll}
          0.5\: т.\: продукта\: I, & \mbox {}\\
          0.5\: т.\: продукта\: II, & \mbox { } \\
\end{array} \right. $ - с одной тонны

$Conv\_pow = 50\: тыс.\: тонн/день$ - мощность переработки конвертера

$Conv\_pr =  0.25\: руб./т$ - стоимость переработки конвертера

$Filter =  0.10\: руб./т$ - стоимость фильтрации первого продукта

## Реализация продукции

$sell\_II = 3.80\: руб./т.$ - неограниченная продажа продукта $II$

$sell\_I_0 =  5.5\: руб./т\:\: limit = 45\: тыс.\:тонн/день$ - продажа $I$

$sell\_I_1 =  5.2\: руб./т\:\: limit = 4\: тыс.\:тонн.день$ - продажа $I$

$sell\_I_2 = 5\: руб./т\:\: unlimited$ - продажа $I$

$pr\_I = 5.75\: руб./т$ - докупка $I$ товара

$pr\_II = 4.00\: руб./т.$ - докупка $II$ товара

$lim1 = 45000\:тыс.\:тонн$

$lim2 = 4000\:тыс.\:тонн$

$min\_lim = 40\: тыс:\: тонн.день$ - минимальный контракт


# Математическая модель

Целевая функция:


$sell\_II\cdot(A\_proc_2\cdot(bA) + B\_proc_2\cdot(bB)-Conv\_proc\cdot(conv)) + sell\_I_0\cdot (sI)\cdot(A\_proc_1\cdot(bA)+B\_proc_1\cdot(bB)+Conv\_proc\cdot(conv))
+sell\_I_1\cdot (sIlim)\cdot(A\_proc_1\cdot(bA)+B\_proc_1\cdot(bB)+Conv\_proc\cdot(conv)) + sell\_I_2\cdot (sIunlim)\cdot(A\_proc_1\cdot(bA)+B\_proc_1\cdot(bB)+Conv\_proc\cdot(conv)) 
-A\_pr\cdot(bA) - B\_pr\cdot(bB) - Conv\_pr\cdot(conv) - Filt\cdot(A\_proc_1\cdot(bA)+B\_proc_1\cdot(bB)) -pr\_I\cdot(bI) - pr\_II\cdot(bII) - proc\_pr\cdot(bA+bB)         $


Ограничения:

$$0\leq bA\leq A\_am$$

$$0\leq bB\leq B\_am$$

$$0\leq bA+bB\leq 25000$$

$$0\leq conv\leq Conv_pow$$

$$0\leq sI + sIlim + sIunlim\leq 1$$

$$0\leq sI$$

$$0\leq sIlim$$

$$0\leq sIlim$$

$$0\leq b1$$

$$0\leq b2$$

$$bA * A\_proc_2 + bB * B\_proc_2+ bII >= conv$$

$$(sI)(A\_proc_1\cdot(bA)+B\_proc_1\cdot(bB)+Conv\_proc\cdot(conv))\leq lim1$$

$$(sIlim)(A\_proc_1\cdot(bA)+B\_proc_1\cdot(bB)+Conv\_proc\cdot(conv))\leq lim2$$

$$(A\_proc_1\cdot(bA)+B\_proc_1\cdot(bB)+Conv\_proc\cdot(conv))\geq min\_lim$$


# Пункт 1
Находим параметры для максимизации прибыли

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from random import randrange
import string
import math
# Коэффиценты целевой функции 

Apr = 3.25
Bpr = 3.40
proc_pr = 0.35
A_proc1 = 0.15
A_proc2 = 0.85
B_proc1 = 0.2
B_proc2 = 0.8
Conv_proc = 0.5
Conv_pr = 0.25
Filt = 0.1
sell_II = 3.8
sell_I0 = 5.5
sell_I1 = 5.1
sell_I2 = 5
pr1 = 5.75
pr2 = 4

#Коэффиценты ограничений

A_am = 100000
B_am = 30000
proc_pow = 100000
Conv_pow = 50000
lim1 = 45000
lim2 = 4000
min_lim = 40000

In [ ]:
#Create a problem
import unicodedata

problem = gp.Model("Factory")
bA =    problem.addVar(lb = 0, ub = 100000,vtype = GRB.CONTINUOUS,name = 'bA')
bB =    problem.addVar(lb = 0, ub = 30000,vtype = GRB.CONTINUOUS,name = 'bB')
conv =  problem.addVar(lb = 0, ub = 50000,vtype = GRB.CONTINUOUS,name = 'conv')
sI =      problem.addVar(lb = 0, ub = 1,vtype = GRB.CONTINUOUS,name = 'sI')
sIlim =   problem.addVar(lb = 0, ub = 1,vtype = GRB.CONTINUOUS,name = 'sIlim')
sIunlim = problem.addVar(lb = 0, ub = 1,vtype = GRB.CONTINUOUS,name = 'sIunlim')
b1 =      problem.addVar(lb = 0,ub=GRB.INFINITY,vtype = GRB.CONTINUOUS,name = 'b1')
b2 =      problem.addVar(lb = 0,ub=GRB.INFINITY,vtype = GRB.CONTINUOUS,name = 'b2')

#Set objective 

problem.setObjective(sell_II * (A_proc2 * bA + B_proc2 * bB - Conv_proc * (conv)) + sell_I0 * (sI) * (A_proc1 * (bA) + B_proc1 * (bB) + Conv_proc * (conv))
+ sell_I1 * (sIlim)*(A_proc1*(bA) + B_proc1 * (bB) + Conv_proc*(conv)) + sell_I2 * (sIunlim)*(A_proc1 * (bA) + B_proc1 * (bB) + Conv_proc * (conv)) 
- Apr * (bA) - Bpr * (bB) - Conv_pr * (conv) - Filt * (A_proc1 * (bA) + B_proc1 * (bB)) - pr1 * (b1) - pr2*(b2) - proc_pr * (bA + bB), GRB.MAXIMIZE)

#Add constrains

problem.addConstr(bA+bB <=proc_pow)
problem.addConstr(sI+sIlim+sIunlim<=1)
problem.addConstr(bA * A_proc2 + bB * B_proc2+ b2 >= conv)
problem.addConstr(sI*(A_proc1 * (bA) + B_proc1*(bB) + Conv_proc*(conv))<=lim1)
problem.addConstr(sIlim * (A_proc1 * (bA) + B_proc1 * (bB) + Conv_proc * (conv))<=lim2)
problem.addConstr((A_proc1 * (bA) + B_proc1 * (bB)+Conv_proc * (conv))>=min_lim)

problem.setParam(GRB.Param.NonConvex,2)
problem.update()
print(problem.display())

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Maximize
   <gurobi.QuadExpr: -0.385 bA + -0.7299999999999999 bB + -2.15 conv + -5.75 b1 + -4.0 b2 + [ 0.825 bA * sI + 0.7649999999999999 bA * sIlim + 0.75 bA * sIunlim + 1.1 bB * sI + 1.02 bB * sIlim + bB * sIunlim + 2.75 conv * sI + 2.55 conv * sIlim + 2.5 conv * sIunlim ]>
Subject To
   R0 : <gurobi.LinExpr: bA + bB> <= 100000.0
   R1 : <gurobi.LinExpr: sI + sIlim + sIunlim> <= 1.0
   R2 : <gurobi.LinExpr: 0.85 bA + 0.8 bB + -1.0 conv + b2> >= 0.0
   R3 : <gurobi.LinExpr: 0.15 bA + 0.2 bB + 0.5 conv> >= 40000.0
    : <gurobi.QuadExpr: 0.0 + [ 0.15 bA * sI + 0.2 bB * sI + 0.5 conv * sI ]> <= 45000.0
    : <gurobi.QuadExpr: 0.0 + [ 0.15 bA * sIlim + 0.2 bB * sIlim + 0.5 conv * sIlim ]> <= 4000.0
Bounds
   0.0 <= bA <= 100000.0
   0.0 <= bB <= 30000.0
   0.0 <= conv <= 50000.0
   0.0 <= sI <= 1.0
   0.0 <= sIlim <= 1.0
   0.0 <= sIunlim <= 1.0
None


In [ ]:
#NonConvex = 2
#problem.setParam("NonConvex",2)
problem.setObjective(total, GRB.MAXIMIZE)
problem.update()  
problem.optimize()

Parameter NonConvex unchanged
   Value: 2  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 4 rows, 8 columns and 12 nonzeros
Model fingerprint: 0x8df81f32
Model has 9 quadratic objective terms
Model has 2 quadratic constraints
Variable types: 8 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  QMatrix range    [1e-01, 5e-01]
  Objective range  [4e-01, 6e+00]
  QObjective range [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+05]
  RHS range        [1e+00, 1e+05]
  QRHS range       [4e+03, 5e+04]
Presolved: 36 rows, 16 columns, 94 nonzeros
Presolved model has 9 bilinear constraint(s)

Continuing optimization...


Cutting planes:
  RLT: 2

Explored 1 nodes (36 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 74000 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.400000000000e+04, best bound 7.400000000000e+04, gap 0.0000%


In [ ]:
if problem.status == GRB.OPTIMAL:
    print('\nIncome: %g' % problem.objVal)


Income: 74000


In [ ]:
problem.getVars()

[<gurobi.Var bA (value 100000.0)>,
 <gurobi.Var bB (value 0.0)>,
 <gurobi.Var conv (value 50000.0)>,
 <gurobi.Var sI (value 1.0)>,
 <gurobi.Var sIlim (value 0.0)>,
 <gurobi.Var sIunlim (value 0.0)>,
 <gurobi.Var b1 (value 0.0)>,
 <gurobi.Var b2 (value 0.0)>]

# Пункт 2
Введем новую переменную $minpr$ и минимизируем её

Наложим ограничение на целевую функцию $F\geq Min\_inc$

$Min\_inc = 50000$

Если в этом пункте использовать целевую функцию из предыдущего пункта, то возникнет произведение из 3 переменных, с которым гуроби не сможет справиться (уже проверил). Поэтому придется поменять мат. модель.

# Введем переменные

$bA$ - количество поставки руды A

$bB$ - количество поставки руды B

$conv$ - количество $II$ продукта отправленного на конвертацию

$sI$ - количество $I$ товара на продажу 

$sIlim$ - покупка до 4 тысяч тонн в день 

$sIunlim$ - безлимитная покупка 

$bI$ - закупка первого товара

$bII$ - закупка второго товара


# Входные данные

## Закуп:
$A\_pr = 3.25\_ руб./т.$ - стоимость покупки руды A

$A\_am = 100\_ тыс. тонн/ день$ - количество поставки руды A в день

$B\_pr = 3.40\_ руб./т.$ - стоимост покупки руды B

$B\_am =  30\_ тыс. тонн/ день $ - количество поставки руды B в день

## Переработка:

$proc\_pow = 100\_ тыс. тонн/ день$ - общая мощность основной переработки

$proc\_pr = 0.35\_ руб./т. $ стоимость общей переработки


$ A\_proc = \left\{
\begin{array}{ll}
          A\_proc1 = 0.15\: т.\: продукта\: I, & \mbox {}\\
          A\_proc2 = 0.85\: т.\: продукта\: II, & \mbox { } \\
\end{array} \right. $ - с одной тонны

$B\_proc = \left\{
\begin{array}{ll}
          B\_proc1 = 0.25\: т.\: продукта\: I, & \mbox {}\\
          B\_proc2 = 0.75\: т.\: продукта\: II, & \mbox { } \\
\end{array} \right. $ - с одной тонны

$ Conv\_proc = \left\{
\begin{array}{ll}
          0.5\: т.\: продукта\: I, & \mbox {}\\
          0.5\: т.\: продукта\: II, & \mbox { } \\
\end{array} \right. $ - с одной тонны

$Conv\_pow = 50\: тыс.\: тонн/день$ - мощность переработки конвертера

$Conv\_pr =  0.25\: руб./т$ - стоимость переработки конвертера

$Filter =  0.10\: руб./т$ - стоимость фильтрации первого продукта


## Реализация продукции

$sell\_II = 3.80\: руб./т.$ - неограниченная продажа продукта $II$

$sell\_I_0 =  5.5\: руб./т\:\: limit = 45\: тыс.\:тонн/день$ - продажа $I$

$sell\_I_1 =  5.2\: руб./т\:\: limit = 4\: тыс.\:тонн.день$ - продажа $I$

$sell\_I_2 = 5\: руб./т\:\: unlimited$ - продажа $I$

$pr\_I = 5.75\: руб./т$ - докупка $I$ товара

$pr\_II = 4.00\: руб./т.$ - докупка $II$ товара

$lim1 = 45000\:тыс.\:тонн$

$lim2 = 4000\:тыс.\:тонн$

$min\_lim = 40\: тыс:\: тонн.день$ - минимальный контракт


# Математическая модель

Целевая функция:

$$sell\_II\cdot(A\_proc_2\cdot(bA) + B\_proc_2\cdot(bB)-Conv\_proc\cdot(conv)) + sell\_I_0\cdot (sI)
+sell\_I_1\cdot (sIlim) + sell\_I_2\cdot (sIunlim) 
-A\_pr\cdot(bA) - B\_pr\cdot(bB) - Conv\_pr\cdot(conv) - Filt\cdot(A\_proc_1\cdot(bA)+B\_proc_1\cdot(bB)) -pr\_I\cdot(bI) - pr\_II\cdot(bII) - proc\_pr\cdot(bA+bB)         $$


Ограничения:

$$0\leq bA\leq A\_am$$

$$0\leq bB\leq A\_am$$

$$0\leq bA+bB\leq proc\_pow$$

$$0\leq conv\leq conv\_pow$$

$$ A\_proc1\cdot bA + B\_proc1\cdot bB + Conv\_proc\cdot conv + b1 = sI + sIlim + sIunlim$$

$$0\leq sI \leq lim1$$

$$0\leq sIlim \leq lim2$$

$$0\leq sIunlim $$

$$0\leq b1$$

$$0\leq b2$$

$$bA * A\_proc_2 + bB * B\_proc_2+ bII >= conv$$

$$(A\_proc_1\cdot(bA)+B\_proc_1\cdot(bB)+Conv\_proc\cdot(conv))\geq min\_lim$$


In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from random import randrange
import string
import math
# Коэффиценты целевой функции 
Apr = 3.25
Bpr = 3.40
proc_pr = 0.35
A_proc1 = 0.15
A_proc2 = 0.85
B_proc1 = 0.25
B_proc2 = 0.75
Conv_proc = 0.5
Conv_pr = 0.25
Filt = 0.1
sell_II = 3.8
pr1 = 5.75
pr2 = 4

#Коэффиценты ограничений

A_am = 100000
B_am = 30000
proc_pow = 100000
Conv_pow = 50000
lim1 = 45000
lim2 = 4000
min_lim = 40000
Min_inc = 50000

In [ ]:
#Create a problem

problem2 = gp.Model("Factory")
bA =    problem2.addVar(lb = 0, ub = 100000,vtype = GRB.CONTINUOUS,name = "bA")
bB =    problem2.addVar(lb = 0, ub = 30000,vtype = GRB.CONTINUOUS,name = "bB")
conv =  problem2.addVar(lb = 0, ub = 50000,vtype = GRB.CONTINUOUS,name = "conv")
sI =      problem2.addVar(lb = 0, ub = 45000,vtype = GRB.CONTINUOUS,name = "sI")
sIlim =   problem2.addVar(lb = 0, ub = 4000,vtype = GRB.CONTINUOUS,name = "sIlim")
sIunlim = problem2.addVar(lb = 0, ub = GRB.INFINITY,vtype = GRB.CONTINUOUS,name = "sIunlim")
b1 =      problem2.addVar(lb = 0,ub=GRB.INFINITY,vtype = GRB.CONTINUOUS,name = "b1")
b2 =      problem2.addVar(lb = 0,ub=GRB.INFINITY,vtype = GRB.CONTINUOUS,name = "b2")
minpr =  problem2.addVar(lb = 0,ub=GRB.INFINITY,vtype = GRB.CONTINUOUS,name = "minpr")
#Set objective 



#Add constrains

problem2.addConstr(bA+bB <=proc_pow)
problem2.addConstr(A_proc1 * bA + B_proc1 * bB + Conv_proc * conv + b1 , GRB.EQUAL, sI+sIlim+sIunlim)
problem2.addConstr(bA * A_proc2 + bB * B_proc2+ b2 >= conv)
problem2.addConstr((A_proc1 * (bA) + B_proc1 * (bB)+Conv_proc * (conv) + b1)>=min_lim)
problem2.addConstr(sell_II * (A_proc2 * bA + B_proc2 * bB - Conv_proc * (conv) + b2) +  (sI)* minpr 
+ (sIlim)*minpr + (sIunlim)*minpr 
- Apr * (bA) - Bpr * (bB) - Conv_pr * (conv) - Filt * (A_proc1 * (bA) + B_proc1 * (bB)) - 
                     pr1 * (b1) - pr2*(b2) - proc_pr * (bA + bB) >= Min_inc)

problem2.setObjective(minpr, GRB.MINIMIZE)

problem2.update()
print(problem2.display())

Minimize
   <gurobi.LinExpr: minpr>
Subject To
   R0 : <gurobi.LinExpr: bA + bB> <= 100000.0
   R1 : <gurobi.LinExpr: 0.15 bA + 0.2 bB + 0.5 conv + -1.0 sI + -1.0 sIlim + -1.0 sIunlim + b1> = 0.0
   R2 : <gurobi.LinExpr: 0.85 bA + 0.8 bB + -1.0 conv + b2> >= 0.0
   R3 : <gurobi.LinExpr: 0.15 bA + 0.2 bB + 0.5 conv + b1> >= 40000.0
    : <gurobi.QuadExpr: -0.3850000000000001 bA + -0.73 bB + -2.15 conv + -5.75 b1 + -0.20000000000000018 b2 + [ sI * minpr + sIlim * minpr + sIunlim * minpr ]> >= 50000.0
Bounds
   0.0 <= bA <= 100000.0
   0.0 <= bB <= 30000.0
   0.0 <= conv <= 50000.0
   0.0 <= sI <= 45000.0
   0.0 <= sIlim <= 4000.0
None


In [ ]:
problem2.setParam(GRB.Param.NonConvex,2)
problem2.setParam(GRB.Param.TimeLimit,6)
problem2.optimize()


Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter TimeLimit to 6.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 4 rows, 9 columns and 17 nonzeros
Model fingerprint: 0x02ed89b8
Model has 1 quadratic constraint
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [2e-01, 6e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+03, 1e+05]
  RHS range        [4e+04, 1e+05]
  QRHS range       [5e+04, 5e+04]

Continuous model is non-convex -- solving as a MIP.

Presolve time: 0.00s
Presolved: 11 rows, 13 columns, 37 nonzeros
Presolved model has 3 bilinear constraint(s)
Variable types: 13 continuous, 0 integer (0 binary)

Root relaxation: objective 0.000000e+00, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth In

In [ ]:
if min_price_problem.status == GRB.OPTIMAL:
    print('\nMin Price: %g' % min_price_problem.objVal)
min_price_problem.getVars()


Min Price: 4.9


[<gurobi.Var A (value 100000.0)>,
 <gurobi.Var B (value 0.0)>,
 <gurobi.Var C (value 50000.0)>,
 <gurobi.Var Z[0] (value 0.0)>,
 <gurobi.Var Z[1] (value 0.0)>,
 <gurobi.Var sum1 (value 40000.00000000002)>,
 <gurobi.Var sum2 (value 0.0)>,
 <gurobi.Var sum3 (value 0.0)>,
 <gurobi.Var sell1 (value 4.899999999999998)>]

# Пункт 3
Как изменится оптимальный план, если мощность фильтра ограничена величиной 10 тыс. тонн?
$Filt\_lim = 10000$

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from random import randrange
import string
import math
# Коэффиценты целевой функции 

Apr = 3.25
Bpr = 3.40
proc_pr = 0.35
A_proc1 = 0.15
A_proc2 = 0.85
B_proc1 = 0.25
B_proc2 = 0.75
Conv_proc = 0.5
Conv_pr = 0.25
Filt = 0.1
sell_II = 3.8
sell_I0 = 5.5
sell_I1 = 5.1
sell_I2 = 5
pr1 = 5.75
pr2 = 4

#Коэффиценты ограничений

A_am = 100000
B_am = 30000
proc_pow = 100000
Conv_pow = 50000
lim1 = 45000
lim2 = 4000
min_lim = 40000
Min_inc = 50000
Filt_lim = 10000

In [ ]:
problem3 = gp.Model("Factory")
bA =    problem3.addVar(lb = 0, ub = 100000,vtype = GRB.CONTINUOUS,name = "bA")
bB =    problem3.addVar(lb = 0, ub = 30000,vtype = GRB.CONTINUOUS,name = "bB")
conv =  problem3.addVar(lb = 0, ub = 50000,vtype = GRB.CONTINUOUS,name = "conv")
sI =      problem3.addVar(lb = 0, ub = 45000,vtype = GRB.CONTINUOUS,name = "sI")
sIlim =   problem3.addVar(lb = 0, ub = 4000,vtype = GRB.CONTINUOUS,name = "sIlim")
sIunlim = problem3.addVar(lb = 0, ub = GRB.INFINITY,vtype = GRB.CONTINUOUS,name = "sIunlim")
b1 =      problem3.addVar(lb = 0,ub=GRB.INFINITY,vtype = GRB.CONTINUOUS,name = "b1")
b2 =      problem3.addVar(lb = 0,ub=GRB.INFINITY,vtype = GRB.CONTINUOUS,name = "b2")

#Set objective 



#Add constrains

problem3.addConstr(bA+bB <=proc_pow)
problem3.addConstr(A_proc1 * bA + B_proc1 * bB + Conv_proc * conv + b1 , GRB.EQUAL, sI+sIlim+sIunlim)
problem3.addConstr(bA * A_proc2 + bB * B_proc2+ b2 >= conv)
problem3.addConstr((A_proc1 * (bA) + B_proc1 * (bB)+Conv_proc * (conv) + b1)>=min_lim)
problem3.addConstr(A_proc1*(bA) + B_proc1 * (bB)  <= Filt_lim)

problem3.setObjective(sell_II * (A_proc2 * bA + B_proc2 * bB - Conv_proc * (conv) + b2) +  (sI)*sell_I0  
+ (sIlim)*sell_I1 + (sIunlim)*sell_I2 
- Apr * (bA) - Bpr * (bB) - Conv_pr * (conv) - Filt * (A_proc1 * (bA) + B_proc1 * (bB)) - 
                     pr1 * (b1) - pr2*(b2) - proc_pr * (bA + bB), GRB.MAXIMIZE)

problem3.update()
print(problem3.display())

Maximize
   <gurobi.LinExpr: -0.385 bA + -0.9250000000000003 bB + -2.15 conv + 5.5 sI + 5.1 sIlim + 5.0 sIunlim + -5.75 b1 + -0.20000000000000018 b2>
Subject To
   R0 : <gurobi.LinExpr: bA + bB> <= 100000.0
   R1 : <gurobi.LinExpr: 0.15 bA + 0.25 bB + 0.5 conv + -1.0 sI + -1.0 sIlim + -1.0 sIunlim + b1> = 0.0
   R2 : <gurobi.LinExpr: 0.85 bA + 0.75 bB + -1.0 conv + b2> >= 0.0
   R3 : <gurobi.LinExpr: 0.15 bA + 0.25 bB + 0.5 conv + b1> >= 40000.0
   R4 : <gurobi.LinExpr: 0.15 bA + 0.25 bB> <= 10000.0
Bounds
   0.0 <= bA <= 100000.0
   0.0 <= bB <= 30000.0
   0.0 <= conv <= 50000.0
   0.0 <= sI <= 45000.0
   0.0 <= sIlim <= 4000.0
None


In [ ]:
problem3.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 5 rows, 8 columns and 19 nonzeros
Model fingerprint: 0x4d1f1a78
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [2e-01, 6e+00]
  Bounds range     [4e+03, 1e+05]
  RHS range        [1e+04, 1e+05]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 4 rows, 7 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+31   1.000000e+30   4.000000e+01      0s
       3    5.8083333e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  5.808333333e+04


In [ ]:
if problem3.status == GRB.OPTIMAL:
    print('\nIncome: %g' % problem3.objVal)
problem3.getVars()


Income: 58083.3


[<gurobi.Var bA (value 66666.66666666667)>,
 <gurobi.Var bB (value 0.0)>,
 <gurobi.Var conv (value 50000.0)>,
 <gurobi.Var sI (value 40000.0)>,
 <gurobi.Var sIlim (value 0.0)>,
 <gurobi.Var sIunlim (value 0.0)>,
 <gurobi.Var b1 (value 5000.0)>,
 <gurobi.Var b2 (value 0.0)>]

# Пункт 4
Как изменится оптимальный план производства, если фильтр выйдет из строя, и продукт I можно будет продавать только по цене 5 руб./т.?
$Filt_pr = 0$

$sell\_I0 = sell\_I1 = sell\_I2 = 5\: руб.\: тонн$




In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from random import randrange
import string
import math
# Коэффиценты целевой функции 

Apr = 3.25
Bpr = 3.40
proc_pr = 0.35
A_proc1 = 0.15
A_proc2 = 0.85
B_proc1 = 0.25
B_proc2 = 0.75
Conv_proc = 0.5
Conv_pr = 0.25
Filt = 0
sell_II = 3.8
sell_I0 = 5
sell_I1 = 5
sell_I2 = 5
pr1 = 5.75
pr2 = 4

#Коэффиценты ограничений

A_am = 100000
B_am = 30000
proc_pow = 100000
Conv_pow = 50000
lim1 = 45000
lim2 = 4000
min_lim = 40000
Min_inc = 50000


In [ ]:
problem4 = gp.Model("Factory")
bA =    problem4.addVar(lb = 0, ub = 100000,vtype = GRB.CONTINUOUS,name = "bA")
bB =    problem4.addVar(lb = 0, ub = 30000,vtype = GRB.CONTINUOUS,name = "bB")
conv =  problem4.addVar(lb = 0, ub = 50000,vtype = GRB.CONTINUOUS,name = "conv")
sI =      problem4.addVar(lb = 0, ub = 45000,vtype = GRB.CONTINUOUS,name = "sI")
sIlim =   problem4.addVar(lb = 0, ub = 4000,vtype = GRB.CONTINUOUS,name = "sIlim")
sIunlim = problem4.addVar(lb = 0, ub = GRB.INFINITY,vtype = GRB.CONTINUOUS,name = "sIunlim")
b1 =      problem4.addVar(lb = 0,ub=GRB.INFINITY,vtype = GRB.CONTINUOUS,name = "b1")
b2 =      problem4.addVar(lb = 0,ub=GRB.INFINITY,vtype = GRB.CONTINUOUS,name = "b2")

#Set objective 



#Add constrains

problem4.addConstr(bA+bB <=proc_pow)
problem4.addConstr(A_proc1 * bA + B_proc1 * bB + Conv_proc * conv + b1 , GRB.EQUAL, sI+sIlim+sIunlim)
problem4.addConstr(bA * A_proc2 + bB * B_proc2+ b2 >= conv)
problem4.addConstr((A_proc1 * (bA) + B_proc1 * (bB) + Conv_proc * (conv) + b1)>=min_lim)


problem4.setObjective(sell_II * (A_proc2 * bA + B_proc2 * bB - Conv_proc * (conv) + b2) +  (sI) * sell_I0  
+ (sIlim)*sell_I1 + (sIunlim)*sell_I2 
- Apr * (bA) - Bpr * (bB) - Conv_pr * (conv) - Filt * (A_proc1 * (bA) + B_proc1 * (bB)) - 
                     pr1 * (b1) - pr2*(b2) - proc_pr * (bA + bB), GRB.MAXIMIZE)

problem4.update()
print(problem4.display())

Maximize
   <gurobi.LinExpr: -0.37 bA + -0.9000000000000002 bB + -2.15 conv + 5.0 sI + 5.0 sIlim + 5.0 sIunlim + -5.75 b1 + -0.20000000000000018 b2>
Subject To
   R0 : <gurobi.LinExpr: bA + bB> <= 100000.0
   R1 : <gurobi.LinExpr: 0.15 bA + 0.25 bB + 0.5 conv + -1.0 sI + -1.0 sIlim + -1.0 sIunlim + b1> = 0.0
   R2 : <gurobi.LinExpr: 0.85 bA + 0.75 bB + -1.0 conv + b2> >= 0.0
   R3 : <gurobi.LinExpr: 0.15 bA + 0.25 bB + 0.5 conv + b1> >= 40000.0
Bounds
   0.0 <= bA <= 100000.0
   0.0 <= bB <= 30000.0
   0.0 <= conv <= 50000.0
   0.0 <= sI <= 45000.0
   0.0 <= sIlim <= 4000.0
None


In [ ]:
problem4.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 4 rows, 8 columns and 17 nonzeros
Model fingerprint: 0x8900f375
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [2e-01, 6e+00]
  Bounds range     [4e+03, 1e+05]
  RHS range        [4e+04, 1e+05]
Presolve removed 4 rows and 8 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5500000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  5.550000000e+04


if problem4.status == GRB.OPTIMAL:
    print('\nIncome: %g' % problem4.objVal)
problem4.getVars()

# П 1

In [ ]:
import gurobipy as gp   #new 1
from gurobipy import GRB
import numpy as np
from random import randrange
import string
import math

In [ ]:
price_A = 3.25
price_B = 3.4
cost_1 = 5.75
cost_2 = 4

cost_Proccesing = 0.35
cost_Konv = 0.25
cost_Filter = 0.1

max_A = 100000
max_B = 30000
max_P = 100000
max_K = 50000

A1 = 0.15
A2 = 0.85
B1 = 0.25
B2 = 0.75
k1 = 0.5
k2 = 0.5

sale_a1 = 5.5
sale_a2 = 5.2
sale_a3 = 5
sale_b = 3.8
sale_min = 40000

q1 = 45000
q2 = 4000

In [ ]:
problem = gp.Model("Factory_Problem_1")
a = problem.addVar(vtype=GRB.INTEGER, lb=0.0, ub = max_A, name = 'a')
b = problem.addVar(vtype=GRB.INTEGER, lb=0.0, ub = max_B, name = 'b')
d1 = problem.addVar(vtype=GRB.INTEGER,lb=0.0, ub = GRB.INFINITY, name = 'd1')
d2 = problem.addVar(vtype=GRB.INTEGER,lb=0.0, ub = GRB.INFINITY, name = 'd2')
k = problem.addVar(vtype=GRB.INTEGER,lb=0.0, ub = max_K, name = 'k')
r1 = problem.addVar(lb=0.0, ub = GRB.INFINITY, name = 'r1')
r2 = problem.addVar(lb=0.0, ub = GRB.INFINITY, name = 'r2')

In [ ]:
problem.addConstr(a+b <= max_P)
problem.addConstr(k <= a*A2+b*B2+d2)
problem.addConstr(r1 >= sale_min)
problem.addConstr(r1 == a*A1+b*B1+k*k1+d1)
problem.addConstr(r2 == a*A2+b*B2+k*(k2-1)+d2)

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
sgn1 = problem.addVar(name="sgn1", vtype = GRB.BINARY)
problem.addGenConstrIndicator(sgn1, 0,(q1-r1) <= 0)
problem.addGenConstrIndicator(sgn1, 1,(q1-r1) >= np.finfo(float).eps)

sgn2 = problem.addVar(name="sgn2", vtype = GRB.BINARY)
problem.addGenConstrIndicator(sgn2, 0,(q1+q2-r1) <= 0)
problem.addGenConstrIndicator(sgn2, 1,(q1+q2-r1) >= np.finfo(float).eps)


sgn3 = problem.addVar(name="sgn3", vtype = GRB.BINARY)
problem.addGenConstrIndicator(sgn3, 0,(r1-(q1+q2)) <= 0)
problem.addGenConstrIndicator(sgn3, 1,(r1-(q1+q2)) >= np.finfo(float).eps)

<gurobi.GenConstr *Awaiting Model Update*>

In [ ]:
part1 = sgn1*r1*sale_a1
part2 = (sgn2-sgn1)*(q1*sale_a1+(r1-q1)*sale_a2-r1*sale_a1)
part3 = sgn3*(q1*sale_a1+q2*sale_a2+(r1-(q1+q2))*sale_a3)
part4 = r2*sale_b-(a+b)*cost_Proccesing - k*cost_Konv
part5 = -(a*A1+b*B1)*cost_Filter-d1*cost_1-d2*cost_2-a*price_A-b*price_B

In [ ]:
total = part1+part2+part3+part4+part5

problem.setObjective(total, GRB.MAXIMIZE)
problem.update()   

In [ ]:
problem.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 5 rows, 10 columns and 17 nonzeros
Model fingerprint: 0xa0bbcf13
Model has 3 quadratic objective terms
Model has 6 general constraints
Variable types: 2 continuous, 8 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [3e-01, 2e+04]
  QObjective range [6e-01, 1e+01]
  Bounds range     [1e+00, 1e+05]
  RHS range        [4e+04, 1e+05]
Presolve added 4 rows and 2 columns
Presolve time: 0.27s
Presolved: 15 rows, 21 columns, 41 nonzeros
Presolved model has 9 SOS constraint(s)
Variable types: 10 continuous, 11 integer (6 binary)
Found heuristic solution: objective 74000.000000

Root relaxation: unbounded, 8 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  postponed    0      74000.0000          -      -     -    0s
     0     0  pos

In [ ]:
if problem.status == GRB.OPTIMAL:
    print(f'\nTotal profit : {problem.objVal}')


Total profit : 74300.0


# доп вопрос

In [ ]:
import gurobipy as gp   
from gurobipy import GRB
import numpy as np
from random import randrange
import string
import math

In [ ]:
price_A = 3.25
price_B = 3.4
cost_1 = 5.75
cost_2 = 4

cost_Proccesing = 0.35 + 0.1
cost_Konv = 0.25 + 0.1
cost_Filter = 0.1 + 0.1

max_A = 100000
max_B = 30000
max_P = 100000
max_K = 50000

A1 = 0.15
A2 = 0.85
B1 = 0.25
B2 = 0.75
k1 = 0.5
k2 = 0.5

sale_a1 = 5.5
sale_a2 = 5.2
sale_a3 = 5
sale_b = 3.8
sale_min = 40000

q1 = 45000
q2 = 4000

In [ ]:
problem = gp.Model("Factory_Problem_1")
a = problem.addVar(vtype=GRB.INTEGER, lb=0.0, ub = max_A, name = 'a')
b = problem.addVar(vtype=GRB.INTEGER, lb=0.0, ub = max_B, name = 'b')
d1 = problem.addVar(vtype=GRB.INTEGER,lb=0.0, ub = GRB.INFINITY, name = 'd1')
d2 = problem.addVar(vtype=GRB.INTEGER,lb=0.0, ub = GRB.INFINITY, name = 'd2')
k = problem.addVar(vtype=GRB.INTEGER,lb=0.0, ub = max_K, name = 'k')
r1 = problem.addVar(lb=0.0, ub = GRB.INFINITY, name = 'r1')
r2 = problem.addVar(lb=0.0, ub = GRB.INFINITY, name = 'r2')

In [ ]:
problem.addConstr(a+b <= max_P)
problem.addConstr(k <= a*A2+b*B2+d2)
problem.addConstr(r1 >= sale_min)
problem.addConstr(r1 == a*A1+b*B1+k*k1+d1)
problem.addConstr(r2 == a*A2+b*B2+k*(k2-1)+d2)

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
sgn1 = problem.addVar(name="sgn1", vtype = GRB.BINARY)
problem.addGenConstrIndicator(sgn1, 0,(q1-r1) <= 0)
problem.addGenConstrIndicator(sgn1, 1,(q1-r1) >= np.finfo(float).eps)

sgn2 = problem.addVar(name="sgn2", vtype = GRB.BINARY)
problem.addGenConstrIndicator(sgn2, 0,(q1+q2-r1) <= 0)
problem.addGenConstrIndicator(sgn2, 1,(q1+q2-r1) >= np.finfo(float).eps)


sgn3 = problem.addVar(name="sgn3", vtype = GRB.BINARY)
problem.addGenConstrIndicator(sgn3, 0,(r1-(q1+q2)) <= 0)
problem.addGenConstrIndicator(sgn3, 1,(r1-(q1+q2)) >= np.finfo(float).eps)

<gurobi.GenConstr *Awaiting Model Update*>

In [ ]:
part1 = sgn1*r1*sale_a1
part2 = (sgn2-sgn1)*(q1*sale_a1+(r1-q1)*sale_a2-r1*sale_a1)
part3 = sgn3*(q1*sale_a1+q2*sale_a2+(r1-(q1+q2))*sale_a3)
part4 = r2*sale_b-(a+b)*cost_Proccesing - k*cost_Konv
part5 = -(a*A1+b*B1)*cost_Filter-d1*cost_1-d2*cost_2-a*price_A-b*price_B

In [ ]:
total = part1+part2+part3+part4+part5

problem.setObjective(total, GRB.MAXIMIZE)
problem.update() 

In [ ]:
problem.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 5 rows, 10 columns and 17 nonzeros
Model fingerprint: 0x72cf9a24
Model has 3 quadratic objective terms
Model has 6 general constraints
Variable types: 2 continuous, 8 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [3e-01, 2e+04]
  QObjective range [6e-01, 1e+01]
  Bounds range     [1e+00, 1e+05]
  RHS range        [4e+04, 1e+05]
Presolve added 4 rows and 2 columns
Presolve time: 0.00s
Presolved: 15 rows, 21 columns, 41 nonzeros
Presolved model has 9 SOS constraint(s)
Variable types: 10 continuous, 11 integer (6 binary)
Found heuristic solution: objective 57500.000000

Root relaxation: unbounded, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  postponed    0      57500.0000          -      -     -    0s
     0     0  pos